In [7]:
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
import numpy as np

In [10]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand
import math

# objective function
def objective(x):
    return x*np.sin(10*np.pi*x)+1


# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    #decoded = list()
    largest = 2 ** n_bits

    # extract the substring
    # convert bitstring to a string of chars
    chars = ''.join([str(s) for s in bitstring])
    # convert string to integer
    integer = int(chars, 2)
    # scale integer to desired range
    value = bounds[0] + (integer / largest) * (bounds[1] - bounds[0])
    # store
    decoded = value
    return decoded


# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k - 1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]


# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1) - 2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]


# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    #print(pop)
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    for gen in range(n_iter):
        decoded = [decode(bounds, n_bits, p) for p in pop]
        scores = [objective(d) for d in decoded]
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                #print(">%d, new best f(%s) = %f" % (gen, decoded[i], scores[i]))
        selected = [selection(pop, scores) for _ in range(n_pop)]
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i + 1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]
        

# define range for input
bounds = [-1.0, 2.0]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 22
# define the population size
n_pop = 20
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
#best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
#print('Done!')
#decoded = decode(bounds, n_bits, best)
#print('f(%s) = %f' % (decoded, -score))

In [11]:
results_x = []
results_y = []

for i in range(10):
    row_x = []
    row_y = []
    for j in range(20):
        best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
        row_x.append(decode(bounds, n_bits, best))
        row_y.append(-score)
    results_x.append(row_x)
    results_y.append(row_y)
    
print('Done!')
    
    

TypeError: 'int' object is not iterable

In [ ]:
plt.scatter(results_x[2], results_y[2])

In [11]:
means_x = [mean(row) for row in results_x]
means_y = [mean(row) for row in results_y]
std_x = [std(row) for row in results_x]
std_y = [std(row) for row in results_y]
print("std:")
print(std_x)

std:
[0.19519429033471922, 0.19485127390309637, 0.19512200180052403, 0.26716525978131905, 0.6028125226039696, 0.12445914762525079, 0.20709884806539616, 0.1949049823475775, 0.21542815170382315, 0.24911783742039867]


In [ ]:
plt.scatter(means_x, means_y)